# **DSCI-100 Project Report**

In [2]:
### Run this cell to load packages
library(tidyverse)
library(repr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [3]:
#run this cell to load in the players and sessions datasets
players  <- read_csv("https://raw.githubusercontent.com/SamuelPastrami/DSCI-PROJECT/refs/heads/main/Data/players.csv")
sessions <- read_csv("https://raw.githubusercontent.com/SamuelPastrami/DSCI-PROJECT/refs/heads/main/Data/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## **Introduction**

#### BACKGROUND

The electronic gaming industry has developed rapidly in recent years, and player participation not only affects game operations, but also provides a rich source of data for academic research. The data used in this project comes from a research group in Computer Science at UBC, led by Frank Wood. They built a Minecraft server to record player behavior data in the virtual world, aiming to explore player behavior patterns and provide a basis for resource management and recruitment strategies.
Due to the resource costs involved in studying server operation, such as software licenses and server capacity, predicting the active time and number of players is crucial to ensure stable server operation. Therefore, this project is a very realistic and challenging data science application scenario. 

#### RESEARCH QUESTIONS

The research team proposed three broad questions, and this project chose the third one: We are interested in demand forecasting, namely, what time windows are most likely to have a large number of simultaneous players. In other words, the question we want to answer is: Can we use the session start times to determine which month is most likely to have a high number of concurrent players on the Minecraft server? 


In [101]:
### run this cell to view the first 6 rows of the players and sessions datasets
head(players)
head(sessions)

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


## **Code Cells being worked on rn (ignore the code for now)**

In [102]:
### run this cell to tidy sessions dataset
convert_date  <- sessions |>
                select(original_start_time) |>                           
                mutate(start_time_seconds = original_start_time/1000) |> #convert from milliseconds to seconds
                mutate(date = as_date(as_datetime(start_time_seconds)))

separate_date <- convert_date |>
                mutate(year  = year(date),
                       month = month(date, label = TRUE),
                       date  = day(date)) |>
                select(year, month, date) |>
                filter(year == 2024)

sessions_tidy <- separate_date |>
                mutate(year  = as.factor(year),
                       month = as.factor(as.character(month)),
                       date  = as.double(date))
head(sessions_tidy)

year,month,date
<fct>,<fct>,<dbl>
2024,Jun,30
2024,Jun,18
2024,Jul,25
2024,Jul,25
2024,May,25
2024,Jun,23
